## 0. ライブラリ・データ読み込み  
　まず初めに使用するライブラリを読み込みます。

In [43]:
# このセルを実行したあとどうすればいいかがわからない方は、別途配布している、「GoogleColabにおけるドライブのマウントの仕方」と、Google Drive上でのフォルダ構造の説明のpdfファイルをご覧ください。

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

import warnings
warnings.filterwarnings('ignore')

In [45]:
# 読み込むデータが格納されたディレクトリのパス，必要に応じて変更の必要あり
# データの読み込み方がわからない方は、別途配布している、GoogleColabにおけるドライブのマウントの仕方、と、Google Drive上でのフォルダ構造の説明のpdfファイルをご覧ください。
path ="/content/drive/MyDrive/GCI/コンペ1/"

df = pd.read_csv(path + 'data/train.csv')
df_test = pd.read_csv(path + 'data/test.csv')

In [46]:
print(df.shape)

(891, 12)


## 1. データの概観・分析・前処理
### 1.1 データの概観
　データを見ていく上で、まず初めにデータのサイズを確認してみましょう。

In [47]:
print('訓練データのデータ数は{}、変数は{}種類です。'.format(df.shape[0], df.shape[1]))
print('テストデータのデータ数は{}、変数は{}種類です'.format(df_test.shape[0], df_test.shape[1]))

訓練データのデータ数は891、変数は12種類です。
テストデータのデータ数は418、変数は11種類です


　訓練データの初めの10データを見てみましょう。

In [48]:
df.head(10)

,PassengerId,Perished,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,1,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,1,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,1,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,1,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,0,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,0,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


　変数名の一覧を見てみましょう。

In [49]:
df.columns

Index(['PassengerId', 'Perished', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

　これらの変数名を、README.ipynbに示された変数の説明と対応付けておきましょう。  

変数 |定義 |備考  
---|---|---
Perished |死亡したかどうか |0 = No, 1 = Yes
Pclass |チケットのクラス |1 = 1st, 2 = 2nd, 3 = 3rd
Name |名前 |
Sex |性別 |
Age	|年齢 |
SibSp |乗船していた兄弟姉妹・配偶者の数	|
Parch |乗船していた親・子供の数	|
Ticket |チケット番号	|
Fare |チケット料金	|
Cabin |キャビン番号	|
embarked |乗船した港	|C = Cherbourg, Q = Queenstown, S = Southampton



## 1.3 データの前処理
　ここでは、機械学習モデルが学習できるようにデータの前処理を行なっていきます。
 
　まずは**欠損値**の補完です。先に見たようにAge、Fare, Cabin、Embarkedの4変数は一部が欠損していました。欠損値の補完には様々な手法があります。平均値や最頻値といった代表値で補完する手法、機械学習モデルで予測して予測値で補完する手法、-9999などの外れ値で補完することによって欠損していたという情報を保持する手法などが挙げられます。
  
　AgeとFareは数値データなので安直に平均値で補完してみましょう。この点で改善の余地は非常に大きいです。例えばNameに含まれる'Miss'と'Mrs'という敬称で平均年齢にズレがあるだろうことは容易に想像できます。余裕があればこのアイデアを実装してみたり、別のアイデアを考えてみたりしてみましょう。名案を思いついたが実装までは難しいという場合でも、他の受講生に実装のアドバイスを求めたり、そのアイデアについて議論してみたりしましょう。

In [50]:
# age = pd.concat([df['Age'], df_test['Age']])
# fare = pd.concat([df['Fare'], df_test['Fare']])

# df['Age'].fillna(age.mean(), inplace=True)
# df_test['Age'].fillna(age.mean(), inplace=True)

# df['Fare'].fillna(fare.mean(), inplace=True)
# df_test['Fare'].fillna(fare.mean(), inplace=True)

# df.isnull().sum() 

# [初心者がKaggle Titanicで上位1.5%(0.83732)以内に入るアプローチ解説_3](https://qiita.com/shiroino11111/items/fb6aa6b7dba2ddc3ce04)

## 客室階層による生存率比較

In [51]:
df['Cabin'] = df['Cabin'].fillna('Unknown') # 客室データが欠損している場合はUnknownを代入
df['Deck'] =df['Cabin'].str.get(0) #Cabin（部屋番号）の頭文字（０番目の文字）取得

df_test['Cabin'] = df_test['Cabin'].fillna('Unknown') # 客室データが欠損している場合はUnknownを代入
df_test['Deck'] =df_test['Cabin'].str.get(0) #Cabin（部屋番号）の頭文字（０番目の文字）取得

df.isnull().sum()

PassengerId      0
Perished         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin            0
Embarked         2
Deck             0
dtype: int64

## チケット

In [52]:
# チケットの文字数による生存率比較
Ticket_Count = dict(df['Ticket'].value_counts()) # チケットの文字数でグループ分け
df['TicketGroup'] = df['Ticket'].apply(lambda x:Ticket_Count[x]) # グループの振り分け

print(Ticket_Count)

Ticket_Count_test = dict(df_test['Ticket'].value_counts()) # チケットの文字数でグループ分け
df_test['TicketGroup'] = df_test['Ticket'].apply(lambda x:Ticket_Count_test[x]) # グループの振り分け

# チケットの文字数での生存率が高いグループと低いグループの２つに分ける。
# 高ければ２，低ければ１を代入
def Ticket_Label(s):
    if (s >= 2) & (s <= 4): # 文字数での生存率が高いグループ
        return 2
    elif ((s > 4) & (s <= 8)) | (s == 1): # 文字数での生存率が低いグループ
        return 1
    elif (s > 8):
        return 0

df['TicketGroup'] = df['TicketGroup'].apply(Ticket_Label)
df_test['TicketGroup'] = df_test['TicketGroup'].apply(Ticket_Label)

{'347082': 7, 'CA. 2343': 7, '1601': 7, '3101295': 6, 'CA 2144': 6, '347088': 6, 'S.O.C. 14879': 5, '382652': 5, 'LINE': 4, 'PC 17757': 4, '17421': 4, '349909': 4, '113760': 4, '4133': 4, '113781': 4, 'W./C. 6608': 4, '2666': 4, '19950': 4, '347077': 4, 'C.A. 31921': 3, 'PC 17572': 3, 'C.A. 34651': 3, '363291': 3, 'F.C.C. 13529': 3, '345773': 3, '248727': 3, '24160': 3, '29106': 3, 'SC/Paris 2123': 3, '35273': 3, '371110': 3, '230080': 3, 'PC 17760': 3, '239853': 3, 'PC 17582': 3, '347742': 3, '110152': 3, '13502': 3, '110413': 3, 'PC 17755': 3, 'PC 17558': 2, '237736': 2, '17474': 2, 'PC 17758': 2, 'PP 9549': 2, 'S.O./P.P. 3': 2, 'P/PP 3381': 2, 'PC 17485': 2, '2668': 2, '2627': 2, 'PC 17604': 2, '2653': 2, '2665': 2, '113798': 2, '31027': 2, '2908': 2, 'W./C. 6607': 2, 'WE/P 5735': 2, '35281': 2, '113789': 2, '244367': 2, '231919': 2, '110465': 2, 'PC 17477': 2, '2699': 2, '367226': 2, '113806': 2, '230136': 2, '111361': 2, '113505': 2, '113803': 2, '370365': 2, '364849': 2, '358585'

## 敬称

In [53]:
# s="Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)"
# s.split(',')[1].split('.')[0].strip()

In [54]:
# 'Honorifics'(敬称)による特徴別に分ける
df['Honorifics'] = df['Name'].apply(lambda x:x.split(',')[1].split('.')[0].strip()) #敬称(','と'.'の間の単語)を抽出
df_test['Honorifics'] = df_test['Name'].apply(lambda x:x.split(',')[1].split('.')[0].strip()) #敬称(','と'.'の間の単語)を抽出

# 敬称をグループ分け
# 例：'Capt', 'Col', 'Major', 'Dr', 'Rev'は'Officer'とする
Honorifics_Dict = {}
Honorifics_Dict.update(dict.fromkeys(['Capt', 'Col', 'Major', 'Dr', 'Rev'], 'Officer'))
Honorifics_Dict.update(dict.fromkeys(['Don', 'Sir', 'the Countess', 'Dona', 'Lady'], 'Royalty'))
Honorifics_Dict.update(dict.fromkeys(['Mme', 'Ms', 'Mrs'], 'Mrs'))
Honorifics_Dict.update(dict.fromkeys(['Mlle', 'Miss'], 'Miss'))
Honorifics_Dict.update(dict.fromkeys(['Mr'], 'Mr'))
Honorifics_Dict.update(dict.fromkeys(['Master','Jonkheer'], 'Master'))
print(Honorifics_Dict)
df['Honorifics'] =df['Honorifics'].map(Honorifics_Dict)
df_test['Honorifics'] =df_test['Honorifics'].map(Honorifics_Dict)

"""敬称一覧
Mr：男 , Master：男の子, Jonkheer：オランダ貴族(男),
Mlle：マドモワゼル (フランス未婚女性), Miss：未婚女性、女の子, Mme：マダム(フランス既婚女性), Ms：女性(未婚・既婚問わず), Mrs：既婚女性, 
Don：男(スペイン), Sir：男(イギリス), the Countess：伯爵夫人, Dona：既婚女性(スペイン), Lady：既婚女性(イギリス),
Capt：船長, Col：大佐, Major：軍人, Dr：医者, Rev：聖職者や牧師
"""

{'Capt': 'Officer', 'Col': 'Officer', 'Major': 'Officer', 'Dr': 'Officer', 'Rev': 'Officer', 'Don': 'Royalty', 'Sir': 'Royalty', 'the Countess': 'Royalty', 'Dona': 'Royalty', 'Lady': 'Royalty', 'Mme': 'Mrs', 'Ms': 'Mrs', 'Mrs': 'Mrs', 'Mlle': 'Miss', 'Miss': 'Miss', 'Mr': 'Mr', 'Master': 'Master', 'Jonkheer': 'Master'}


'敬称一覧\nMr：男 , Master：男の子, Jonkheer：オランダ貴族(男),\nMlle：マドモワゼル (フランス未婚女性), Miss：未婚女性、女の子, Mme：マダム(フランス既婚女性), Ms：女性(未婚・既婚問わず), Mrs：既婚女性, \nDon：男(スペイン), Sir：男(イギリス), the Countess：伯爵夫人, Dona：既婚女性(スペイン), Lady：既婚女性(イギリス),\nCapt：船長, Col：大佐, Major：軍人, Dr：医者, Rev：聖職者や牧師\n'

##  'Age'(年齢)の欠損値の見直し

In [55]:
## 年齢の欠損値を予測して代入
# 年齢予測のために使う項目を抽出し、ダミー変数を作成
age = df[['Age','Pclass','Sex','Honorifics']]
age_dummies = pd.get_dummies(age)


age_test = df_test[['Age','Pclass','Sex','Honorifics']]
age_dummies_test = pd.get_dummies(age_test)


In [56]:
from sklearn.ensemble import RandomForestRegressor
# ageが分かるものと欠損しているものに分ける
known_age = age_dummies[age_dummies.Age.notnull()].values
null_age = age_dummies[age_dummies.Age.isnull()].values

null_age_test = age_dummies_test[age_dummies_test.Age.isnull()].values
# 特徴量と正解データに分ける
age_X = known_age[:, 1:]
age_y = known_age[:, 0]

# 年齢予測モデルを作成し、予測した値を代入
rf = RandomForestRegressor()
rf.fit(age_X, age_y)
pred_Age = rf.predict(null_age[:, 1:])
pred_Age_test=rf.predict(null_age_test[:,1:])
df.loc[(df.Age.isnull()),'Age'] = pred_Age
df_test.loc[(df_test.Age.isnull()),'Age'] = pred_Age_test
df.isnull().sum()

PassengerId    0
Perished       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       2
Deck           0
TicketGroup    0
Honorifics     0
dtype: int64

## 'Embarked'(出港地)の欠損値の見直し

In [57]:
# 'Embarked'の欠損値にCを代入
df['Embarked'] = df['Embarked'].fillna('C')
df_test['Embarked'] = df_test['Embarked'].fillna('C')

In [58]:
df

,PassengerId,Perished,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Deck,TicketGroup,Honorifics
0,1,1,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,A/5 21171,7.2500,Unknown,S,U,1,Mr
1,2,0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000000,1,0,PC 17599,71.2833,C85,C,C,1,Mrs
2,3,0,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,STON/O2. 3101282,7.9250,Unknown,S,U,1,Miss
3,4,0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,113803,53.1000,C123,S,C,2,Mrs
4,5,1,3,"Allen, Mr. William Henry",male,35.000000,0,0,373450,8.0500,Unknown,S,U,1,Mr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,1,2,"Montvila, Rev. Juozas",male,27.000000,0,0,211536,13.0000,Unknown,S,U,1,Officer
887,888,0,1,"Graham, Miss. Margaret Edith",female,19.000000,0,0,112053,30.0000,B42,S,B,1,Miss
888,889,1,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,16.177522,1,2,W./C. 6607,23.4500,Unknown,S,U,2,Miss
889,890,0,1,"Behr, Mr. Karl Howell",male,26.000000,0,0,111369,30.0000,C148,C,C,1,Mr


## 'Fare'(料金)

In [59]:
# 'Fare'(料金)が欠損しているデータを表示
df[df['Fare'].isnull()]

# 'Pclass'(チケットクラス)が3で、'Embarked'(出港地)が'S'の中央値を代入
fare_median=df[(df['Embarked'] == "S") & (df['Pclass'] == 3)].Fare.median()
df['Fare']=df['Fare'].fillna(fare_median)
df_test['Fare']=df_test['Fare'].fillna(fare_median)


## 家族人数

In [60]:
# 同乗している家族の数
df['FamilySize']=df['SibSp']+df['Parch']+1
df_test['FamilySize']=df_test['SibSp']+df_test['Parch']+1

# 家族数による生存率でグループ分け
def Family_label(s):
    if (s >= 2) & (s <= 4):
        return 2
    elif ((s > 4) & (s <= 7)) | (s == 1):
        return 1
    elif (s > 7):
        return 0
df['FamilyLabel']=df['FamilySize'].apply(Family_label)
df_test['FamilyLabel']=df_test['FamilySize'].apply(Family_label)

In [61]:
df

,PassengerId,Perished,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Deck,TicketGroup,Honorifics,FamilySize,FamilyLabel
0,1,1,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,A/5 21171,7.2500,Unknown,S,U,1,Mr,2,2
1,2,0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000000,1,0,PC 17599,71.2833,C85,C,C,1,Mrs,2,2
2,3,0,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,STON/O2. 3101282,7.9250,Unknown,S,U,1,Miss,1,1
3,4,0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,113803,53.1000,C123,S,C,2,Mrs,2,2
4,5,1,3,"Allen, Mr. William Henry",male,35.000000,0,0,373450,8.0500,Unknown,S,U,1,Mr,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,1,2,"Montvila, Rev. Juozas",male,27.000000,0,0,211536,13.0000,Unknown,S,U,1,Officer,1,1
887,888,0,1,"Graham, Miss. Margaret Edith",female,19.000000,0,0,112053,30.0000,B42,S,B,1,Miss,1,1
888,889,1,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,16.177522,1,2,W./C. 6607,23.4500,Unknown,S,U,2,Miss,4,2
889,890,0,1,"Behr, Mr. Karl Howell",male,26.000000,0,0,111369,30.0000,C148,C,C,1,Mr,1,1


In [62]:
# 使用する変数を抽出
con=pd.concat([df,df_test])

con = con[['Perished','Pclass','Sex','Age','Fare','Embarked','Honorifics','FamilyLabel','Deck','TicketGroup']]
# ダミー変数を作成
dataset_dummies = pd.get_dummies(con)
dataset_dummies

,Perished,Pclass,Age,Fare,FamilyLabel,TicketGroup,Sex_female,Sex_male,Embarked_C,Embarked_Q,...,Honorifics_Royalty,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T,Deck_U
0,1.0,3,22.000000,7.2500,2,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0.0,1,38.000000,71.2833,2,1,1,0,1,0,...,0,0,0,1,0,0,0,0,0,0
2,0.0,3,26.000000,7.9250,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0.0,1,35.000000,53.1000,2,2,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,1.0,3,35.000000,8.0500,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,NaN,3,28.659379,8.0500,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
414,NaN,1,39.000000,108.9000,1,1,1,0,1,0,...,1,0,0,1,0,0,0,0,0,0
415,NaN,3,38.500000,7.2500,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
416,NaN,3,28.659379,8.0500,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1


In [63]:
# データをtrainとtestに分解 
#（ 'Survived'が存在するのがtrain, しないのがtest ）
train_set = dataset_dummies[dataset_dummies['Perished'].notnull()]
test_set = dataset_dummies[dataset_dummies['Perished'].isnull()]

# ベースラインモデルの構築

In [64]:
X = train_set.iloc[:, 1:].values
y = train_set.iloc[:, 0].values

X_test =test_set.iloc[:, 1:].values

In [65]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

## optuna使ってみる
* [Optunaを使ったRandomforestの設定方法](https://qiita.com/motoyuki1963/items/9b5c1c8950728f949add)
* [optuna入門](https://qiita.com/studio_haneya/items/2dc3ba9d7cafa36ddffa)
* [Optunaを使ってみる](https://knknkn.hatenablog.com/entry/2021/06/14/160302)

In [ ]:
!pip install optuna
import optuna
from sklearn.metrics import accuracy_score

In [72]:
# optunaの目的関数を設定する
# https://qiita.com/motoyuki1963/items/9b5c1c8950728f949add
# https://www.kaggle.com/code/funaki/random-forest-classifier-of-funaki-optuna
def objective(trial):
    params = {
    'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
    'n_estimators': trial.suggest_int("n_estimators", 10, 500),
    'max_depth': trial.suggest_int("max_depth", 1,10),
    'min_samples_split': trial.suggest_int("min_samples_split", 2,50),
    'min_samples_leaf': trial.suggest_int('min_samples_leaf',1,10),
    'random_state': 0,
    'verbose' : 0
}

    clf = RandomForestClassifier(**params)

    clf.fit(X_train, y_train)
    return  1-accuracy_score(y_valid, clf.predict(X_valid))


In [ ]:
#optunaで学習
study = optuna.create_study()
study.optimize(objective, n_trials=100)


In [79]:
#optunaで見つけた最適なパラメーターを出力
study.best_params

{'criterion': 'entropy',
 'n_estimators': 446,
 'max_depth': 7,
 'min_samples_split': 49,
 'min_samples_leaf': 5}

In [75]:
model =RandomForestClassifier(**study.best_params)

In [76]:
#モデルにさらに全データを学習
rfc=model.fit(X,y)

In [77]:
lr_pred = rfc.predict(X_test)

In [41]:
 # 読み込むデータが格納されたディレクトリのパス，必要に応じて変更の必要あり
path = "/content/drive/MyDrive/GCI/コンペ1/"

submission = pd.read_csv(path + 'gender_submission.csv')
submission['Perished'] = lr_pred
submission

,PassengerId,Perished
0,892,1.0
1,893,0.0
2,894,1.0
3,895,1.0
4,896,0.0
...,...,...
413,1305,1.0
414,1306,0.0
415,1307,1.0
416,1308,1.0


In [42]:
# Google Drive・Google Colaboratoryで作業する場合
submission.to_csv('submission.csv',index=False)

from google.colab import files
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>